Import


In [3]:

# pip install pandas scikit-learn

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


#dataset

In [4]:
movies = pd.read_csv('movies.csv')       # movieId, title, genres
ratings = pd.read_csv('ratings.csv')     # userId, movieId, rating, timestamp
print(movies.head())
print(ratings.head())


FileNotFoundError: [Errno 2] No such file or directory: 'movies.csv'

In [ ]:
# Create TF-IDF matrix for genres
tfidf = TfidfVectorizer(stop_words='english')
movies['genres'] = movies['genres'].fillna('')
tfidf_matrix = tfidf.fit_transform(movies['genres'])

# Compute similarity score
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Function to recommend movies
def recommend_movies(title, cosine_sim=cosine_sim):
    # Get index of the movie
    idx = movies[movies['title'].str.lower() == title.lower()].index[0]
    
    # Get similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort movies by similarity
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get top 10 similar movies
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    
    return movies['title'].iloc[movie_indices]

# Example
print(recommend_movies("Toy Story (1995)"))


In [ ]:
from sklearn.neighbors import NearestNeighbors

# Create user-item matrix
user_movie_matrix = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Fit model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(user_movie_matrix)

# Find similar users to user 1
distances, indices = model_knn.kneighbors([user_movie_matrix.loc[1]], n_neighbors=6)
print(indices)
